In [1]:
import numpy as np
from deap import base
from deap import creator
from deap import tools
import random
import pandas as pd
from deap import algorithms


# Reading From Excel File

In [2]:
df = pd.read_excel(r"C:\Users\mcf\OneDrive\Desktop\Availability (1).xlsx")

In [3]:
samples = np.asarray(df)

In [4]:
df

,country,Fertilizers,Import&Product,Export,Availability,Use,x
0,Brazil,"Ammonia, anhydrous",24144572.05,399156.28,23745415.77,2571549.00,5931.614778
1,Brazil,Ammonium nitrate (AN),21546416.92,214625.65,21331791.27,13821890.37,31882.001541
2,Brazil,Ammonium sulphate,30988444.53,53840.25,30934604.28,22805149.59,52603.066217
3,Brazil,Calcium ammonium nitrate (CAN) and other mixtu...,2020987.02,10844.03,2010142.99,1164236.84,2685.464848
4,Brazil,Diammonium phosphate (DAP),6948831.81,148854.13,6799977.68,4846157.53,11178.297447
...,...,...,...,...,...,...,...
110,Mexico,Sodium nitrate,32779.02,1030.91,31748.11,0.00,0.000000
111,Mexico,Superphosphates above 35%,622825.23,2467787.77,-1844962.54,811919.38,1699.959390
112,Mexico,"Superphosphates, other",0.00,0.00,0.00,21682383.00,45397.574551
113,Mexico,Urea,21917270.86,303460.75,21613810.11,8931655.72,18700.689238


#### In this part we will convert the fertalize names and country names to numbers

In [5]:
country = pd.factorize(df['country'])[0]
fert = pd.factorize(df['Fertilizers'])[0]

In [6]:
samples[:,0] = country
samples[:,1] = fert

In [7]:
samples.shape

(115, 7)

# EC Code
#### in this part we will use availability and production as a fitness functions per country

In [8]:
creator.create("Fitness", base.Fitness, weights = (1.0,))
creator.create("Individual", np.ndarray, fitness = creator.Fitness)

In [9]:
IND_SIZE = 22
toolbox = base.Toolbox()
toolbox.register("indices", random.sample, range(IND_SIZE), IND_SIZE)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [10]:
def evaluation (individual):
    result1 = 0
    result2 = 0
    for i in range (22):
        temp = individual[i]
        for j in range (115):
            if samples [j,0] == 4 and samples [j,1] == individual[i]:
                result1 += samples[j,4] * (23 - i)
                result2 += samples[j,6] * (23 - i)
    return (result1 + result2),

In [19]:
def mutInv (individual):
    index1 = random.randint(0,12)
    index2 = random.randint(13,21)
    for i in range(index1, index2):
        temp = individual[i]
        individual[i] = individual[index2-i]
        individual[index2-i] = temp
    return individual

In [27]:
toolbox.register("evaluate", evaluation)
toolbox.register("cx_pm", tools.cxPartialyMatched)
toolbox.register("cx_onepoint", tools.cxOnePoint)
toolbox.register("cx_twopoint", tools.cxTwoPoint)
toolbox.register("mut_shuffle", tools.mutShuffleIndexes)
toolbox.register("mut_inverse", mutInv)
# indpb = random.randint(0,10)/10)
#toolbox.register("select", tools.selTournament, tournsize = 10)
toolbox.register("sel_roulette", tools.selRoulette)
toolbox.register("sel_best", tools.selBest)
toolbox.register("sel_worst", tools.selWorst)
toolbox.register("sel_random", tools.selRandom)


In [28]:

def main (selection, CX, mutation, CXPB, MUTPB, tsize, popul):
    res= np.zeros (200)
    toolbox.register("sel_trnmnt", tools.selTournament, tournsize = tsize)
    pop = toolbox.population(n = popul)
    fitnesses = list (map (toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    #CXPB, MUTPB = 0.8, 0.8
    fits = [ind.fitness.values[0] for ind in pop]
    g = 0
    while g < 200:
        
        #print ('--Generation %i---' %g)
        if selection == 'trnmnt':
            offspring = toolbox.sel_trnmnt(pop, len(pop))
        elif selection == 'roulette':
            offspring = toolbox.sel_roulette(pop, len(pop))
        elif selection == 'best':
            offspring = toolbox.sel_best(pop, len(pop))
        elif selection == 'worst':
            offspring = toolbox.sel_worst(pop, len(pop))
        elif selection == 'random':
            offspring = toolbox.sel_random(pop, len(pop))
        
        #toolbox.clone() method ensure that we don’t use a reference to the individuals but an completely independent instance.
        offspring = list(map(toolbox.clone, offspring))
                # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                if CX == 'onepoint':
                    toolbox.cx_onepoint(child1, child2)
                elif CX == 'twopoint':
                    toolbox.cx_twopoint(child1, child2)
                elif CX == 'pmx':
                    toolbox.cx_pm(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                if mutation == 'shuffle':
                    toolbox.mut_shuffle(mutant, random.random())
                elif mutation == 'inverse':
                    toolbox.mut_inverse(mutant)
                del mutant.fitness.values
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        pop[:] = offspring
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
       # print("  Min %s" % min(fits))
       # print("  Max %s" % max(fits))
       # print("  Avg %s" % mean)
       # print("  Std %s" % std)
        res[g] = max(fits)
        g = g + 1
    ind = max(fits)
   # print (ind)
    # you can choose haow many best individuals wanna see by k
    winner = tools.selBest(pop, k = 1)
    return res, winner

In [30]:
Selectlist = ['trnmnt', 'roulette', 'best', 'worst', 'random']
CXlist = ['onepoint', 'twopoint', 'pmx']
mut = ['shuffle', 'inverse']

In [32]:
mbf = np.zeros(50)
Meta_GA = np.zeros(350).reshape(50,7)
for i in range (50):
    Meta_GA[i,1] = random.randint(0,10)/10 #crossover
    Meta_GA[i,0] = random.randint(0,10)/10 #mutation
    Meta_GA[i,2] = random.randint(1,100) #tournment size
    Meta_GA[i,3] = random.randint(100,200)#population
    Meta_GA[i,4] = random.randint(0,4)#selection
    Meta_GA[i,5] = random.randint(0,2)#crossover
    Meta_GA[i,6] = random.randint(0,1)#mutation
    res, win = main (Selectlist[int(Meta_GA[i,4])], CXlist[int(Meta_GA[i,5])],mut[int(Meta_GA[i,6])]
                     ,Meta_GA[i,0], Meta_GA[i,1], int(Meta_GA[i,2]), int(Meta_GA[i,3]))
    #main (selection, CX, mutation, CXPB, MUTPB, tsize, popul)
    mbf[i] = np.mean(res, axis = None)

In [53]:
Meta_GA

array([[5.00e-01, 7.00e-01, 2.20e+01, 1.09e+02, 4.00e+00, 1.00e+00,
        1.00e+00],
       [1.00e-01, 1.00e-01, 6.00e+00, 1.85e+02, 0.00e+00, 1.00e+00,
        0.00e+00],
       [1.00e+00, 7.00e-01, 5.00e+00, 1.58e+02, 2.00e+00, 1.00e+00,
        0.00e+00],
       [6.00e-01, 2.00e-01, 5.40e+01, 1.23e+02, 0.00e+00, 0.00e+00,
        0.00e+00],
       [5.00e-01, 5.00e-01, 1.00e+00, 1.06e+02, 2.00e+00, 1.00e+00,
        0.00e+00],
       [6.00e-01, 1.00e+00, 1.40e+01, 1.15e+02, 3.00e+00, 1.00e+00,
        1.00e+00],
       [7.00e-01, 0.00e+00, 1.00e+02, 1.39e+02, 0.00e+00, 2.00e+00,
        1.00e+00],
       [5.00e-01, 6.00e-01, 4.00e+01, 1.11e+02, 3.00e+00, 1.00e+00,
        1.00e+00],
       [0.00e+00, 7.00e-01, 2.00e+01, 1.73e+02, 0.00e+00, 0.00e+00,
        0.00e+00],
       [5.00e-01, 7.00e-01, 4.70e+01, 1.86e+02, 4.00e+00, 0.00e+00,
        0.00e+00],
       [1.00e+00, 2.00e-01, 2.80e+01, 1.95e+02, 3.00e+00, 1.00e+00,
        1.00e+00],
       [5.00e-01, 0.00e+00, 7.20e+01, 1.18e

In [34]:
import matplotlib.pyplot as plt
%matplotlib

plt.plot (mbf)
plt.ylabel ('Mean Best Fitenss')
plt.xlabel ("Different EA's")

Using matplotlib backend: Qt5Agg


Text(0.5, 0, "Different EA's")

In [41]:

arg = np.argsort(mbf)
new = np.zeros(140).reshape(20,7)

for i in range (20):
    new[i,:] = Meta_GA[arg[19-i],:]
new

array([[3.00e-01, 7.00e-01, 7.10e+01, 1.15e+02, 4.00e+00, 0.00e+00,
        0.00e+00],
       [5.00e-01, 9.00e-01, 4.80e+01, 1.67e+02, 0.00e+00, 2.00e+00,
        0.00e+00],
       [0.00e+00, 7.00e-01, 2.00e+01, 1.73e+02, 0.00e+00, 0.00e+00,
        0.00e+00],
       [6.00e-01, 0.00e+00, 6.50e+01, 1.84e+02, 4.00e+00, 1.00e+00,
        0.00e+00],
       [5.00e-01, 0.00e+00, 7.20e+01, 1.18e+02, 4.00e+00, 1.00e+00,
        1.00e+00],
       [8.00e-01, 1.00e-01, 9.90e+01, 1.56e+02, 0.00e+00, 2.00e+00,
        1.00e+00],
       [7.00e-01, 0.00e+00, 1.00e+02, 1.39e+02, 0.00e+00, 2.00e+00,
        1.00e+00],
       [9.00e-01, 1.00e-01, 7.70e+01, 1.75e+02, 2.00e+00, 2.00e+00,
        1.00e+00],
       [0.00e+00, 2.00e-01, 4.70e+01, 1.04e+02, 1.00e+00, 1.00e+00,
        0.00e+00],
       [1.00e+00, 6.00e-01, 5.70e+01, 1.74e+02, 1.00e+00, 2.00e+00,
        0.00e+00],
       [4.00e-01, 6.00e-01, 2.40e+01, 1.78e+02, 1.00e+00, 2.00e+00,
        0.00e+00],
       [0.00e+00, 6.00e-01, 1.50e+01, 1.96e

In [55]:
def cx (k1, k2):
    rand = random.randint(0,6)
    for i in range(rand, 3):
        temp = k1[i]
        k1 [i] = k2[i]
        k2 [i] = temp
    return k1,k2

In [56]:
final = np.zeros(140).reshape(20,7)
for i in range (10):
    index1 = random.randint(0,3)
    index2 = random.randint(0,3)
    final[i,:], final[19-i,:] = cx (new[index1,:], new[index1,:])
    


In [57]:
mbf_final = np.zeros(20)
for i in range(20):
    res1, win = main (Selectlist[int(final[i,4])],CXlist[int(final[i,5])],mut[int(final[i,6])] ,final[i,0], final[i,1], int(final[i,2]), int(final[i,3]))
    #res, win = main (Selectlist[int(Meta_GA[i,4])], CXlist[int(Meta_GA[i,5])],mut[int(Meta_GA[i,6])]
    #                 ,Meta_GA[i,0], Meta_GA[i,1], int(Meta_GA[i,2]), int(Meta_GA[i,3]))
    mbf_final[i] = np.mean (res1)
  #  plt.plot (res1, label = 'new data')
    

In [59]:
%matplotlib
plt.plot(mbf_final, label = 'final result')
plt.plot(mbf, label = 'first result')
plt.legend()

Using matplotlib backend: Qt5Agg


In [58]:
new[0,:]

array([  0.3,   0.7,  71. , 115. ,   4. ,   0. ,   0. ])